<a href="https://colab.research.google.com/github/GruAna/VU/blob/master/m_craft_tess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRAFT + Tesseract

## Packages

In [ ]:
! sudo apt install tesseract-ocr
! pip install craft-text-detector
! pip install pytesseract
! pip install imgaug==0.2.6
! pip install --ignore-installed Pillow==9.0.0

fatal: destination path 'CRAFT-pytorch' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Pillow-9.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from tqdm import tqdm

from craft_text_detector import (
    read_image,
    load_craftnet_model,
    load_refinenet_model,
    get_prediction,
    export_detected_regions,
    export_extra_results,
    empty_cuda_cache
)

import pytesseract
from pytesseract import Output
import cv2 as cv
import numpy as np
import pandas as pd

!cp drive/MyDrive/Colab_Notebooks/VU/utils.py .
from utils import *

## Setup

## Dataset loading

### CTW1500 dataset

 - testing set (500 images)

Get images. Update location of images **manually**.

Comment if not using CTW1500 dataset.

In [ ]:
# whether images are to be resized (skrinking)
resize = False
width = 3000

# path to image directory, get full path to all files
imgs_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/CTW1500/test_images'

(_, _, filenames) = next(os.walk(imgs_dir))
filenames.sort()
list_img_paths = [os.path.join(imgs_dir, file) for file in filenames]
n_imgs = len(list_img_paths)

# load images
original_images = [(cv.imread(file)) for file in list_img_paths]
                        
# shrink images
if resize:
    images = shrink_all(original_images, width)
else:
    images = original_images.copy()

# grayscale
images = [cv.cvtColor(img, cv.COLOR_BGR2GRAY) for img in images]

# # threshold
# patches = [(img.shape[0] // 16) * 2 + 1 for img in images]
# images = [cv.adaptiveThreshold(images[i], 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, patches[i], 5) for i in range(n_imgs)]
# # # closing 
# kernel = cv.getStructuringElement(cv.MORPH_CROSS,(3,3))
# images = [cv.morphologyEx(img, cv.MORPH_CLOSE, kernel) for img in images]    

# Get paths to files with labels. Update location manually
# path to label directory, get full path to all files
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/CTW1500/test_labels'
(_, _, xml_files) = next(os.walk(labels_dir))
xml_files.sort()
list_xml_paths = [os.path.join(labels_dir, file) for file in xml_files]

# Get ground truths from all xml files
ground_truth = []

# if images where resized we need to resize also the coordinates in gt
if resize:
    for i, file in enumerate(list_xml_paths):
        ratio = width / original_images[i].shape[1]
        if ratio > 1:
            ratio = 1
        ground_truth.append(read_gt_ctw_test(file, scaling_ratio=ratio))
else:
    for i, file in enumerate(list_xml_paths):
        ground_truth.append(read_gt_ctw_test(file))

# ground_truth is in the is a list of tuples, where first is the gt word 
# and second is an array of top left and bottom right coordinates
# format: ('text', [[tl,tl],[br,br]])

## Born digital dataset

In [ ]:
# path to image and label directory, get full path to all files
imgs_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/BD/Challenge1_Training_Task12_Images'
labels_dir = '/content/drive/MyDrive/Colab_Notebooks/VU/BD/ch1_training_localization_transcription_gt'

(_, _, filenames) = next(os.walk(imgs_dir))
filenames.sort()
list_img_paths = [os.path.join(imgs_dir, file) for file in filenames]

(_, _, txt_files) = next(os.walk(labels_dir))
txt_files.sort()
list_txt_paths = [os.path.join(labels_dir, file) for file in txt_files]

n_imgs = len(list_img_paths)
n_labels = len(list_txt_paths)

assert n_imgs == n_labels, "Check both ground truth and image files"

# whether images are to be resized (skrinking)
resize = False
width = 3000

# load images
original_images = [(cv.imread(file)) for file in list_img_paths]
# load images = bmp images (detected text)
# from already text detected images, which are provided in Kaist dataset
# original_images = [(cv.imread(file)) for file in list_bmp_paths]
  
# shrink images
if resize:
    images = shrink_all(original_images, width)
else:
    images = original_images.copy()

# grayscale
images = [cv.cvtColor(img, cv.COLOR_BGR2GRAY) for img in images]

# threshold
# patches = [(img.shape[0] // 16) * 2 + 1 for img in images]
# images = [cv.adaptiveThreshold(images[i], 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, patches[i], 5) for i in range(n_imgs)]


# Get ground truths from all xml files
ground_truth = []

# if images where resized we need to resize also the coordinates in gt
if resize:
    for i, file in tqdm(enumerate(list_xml_paths)):
        ratio = width / original_images[i].shape[1]
        if ratio > 1:
            ratio = 1
        ground_truth.append(read_gt_bd(file, scaling_ratio=ratio))
else:
    for i, file in tqdm(enumerate(list_txt_paths)):
        ground_truth.append(read_gt_bd(file))

# ground_truth is in the is a list of tuples, where first is the gt word 
# and second is an array of top left and bottom right coordinates
# format: ('text', [[tl,tl],[br,br]])

410it [05:58,  1.14it/s]


Setup for prediction

In [ ]:
# if GPU is to be used, allow CUDA
use_GPU = True

# set and create output directory if it doesn't exist
detected_dir = 'output'
if not os.path.exists(detected_dir):
    os.makedirs(detected_dir)

# load models
refine_net = load_refinenet_model(cuda=use_GPU)
craft_net = load_craftnet_model(cuda=use_GPU)

custom_config = r'--oem 3 --psm 8'

## Prediction

Run OCR methods - CRAFT and Tesseract

In [ ]:
predicted = []

# loop through all images in image directory
for img in tqdm(images):

    predictions = get_prediction(
        image = img,
        craft_net = craft_net,
        refine_net = refine_net,
        cuda = use_GPU,
        poly = False
    )

    detected_regions = predictions["boxes"]

    image_output = []
    for region in detected_regions:
        coords = bounding_rectangle(region)
        cropped = img[coords[0][1]:coords[1][1], coords[0][0]:coords[1][0]]
        tess_output = pytesseract.image_to_data(cropped, output_type=Output.DICT, config=custom_config)['text']
        words = [t for t in tess_output if len(t) > 0 and not t.isspace()]
        image_output.append((" ".join(words), coords))

    predicted.append(image_output)

# unload models from gpu
if use_GPU:
    empty_cuda_cache()

    

 86%|████████▌ | 353/410 [08:52<01:02,  1.10s/it]

Compare bounding boxes of predicition and ground truth.

Count Intersection over Union (IoU) metric for bounding boxes. Store for all images in a list `iou_images`.

Count Character Error Rate (CER) metric for characters in words. Store for all images in a list `cer_images`.

In [ ]:
iou_images = []
cer_images = []

# loop through images:
for i in range(n_imgs):
    # separate list on columns (iterate through tuples in the list)
    if len(predicted[i]) and len(ground_truth[i]):
        predicted_cols = list(zip(*predicted[i]))
    else:
        iou_images.append(None)
        cer_images.append(None)
        continue
    ground_truth_cols = list(zip(*ground_truth[i]))

    # take only coordinate arrays from list for each images
    pred_boxes = predicted_cols[1]
    gt_boxes = ground_truth_cols[1]
    iou_from_image = iou_image(pred_boxes, gt_boxes)
    iou_text_regions = group_text(iou_from_image)
  
    # take only labels for each image
    pred_labels = predicted_cols[0]
    gt_labels = ground_truth_cols[0]

    # compare corresponding labels
    # comparision is a list of all text regions on one image
    comparision = []
    for observation in iou_text_regions:
        gt_ind = observation[-1]
        pred_ind = observation[1]
        predicted_text = " ".join([pred_labels[i] for i in pred_ind])
        gt_pred_text = (gt_labels[gt_ind], predicted_text)
       

        # comparision for one text region (on one image)
        comparision.append((compare_text_cer(gt_pred_text, special_characters=False, case_sensitive=False, split=False)))

    iou_images.append((iou_text_regions))
    cer_images.append((comparision))

### Metrics
Metrics for each image (average of values of all regions in one image).

IoU (detection) `iou_in_image`

CER (recognition) `cer_in_image`

In [ ]:
iou_in_image = []
cer_in_image = []

for i in range(n_imgs):
    # calculate mean based on results 
    if isinstance(cer_images[i], list):
        length = len(cer_images[i])
        mean_in_regions = average([average(list(zip(*cer_images[i][j]))[2]) for j in range(length) ])
        iou_in_image.append(average(list(zip(*iou_images[i]))[0]))
    else:
        mean_in_regions = None
        iou_in_image.append(None)

    cer_in_image.append(mean_in_regions)

### Overall results for dataset

Specify unique filename and specification.

Specification is an array of first used method, second some useful infromation.

In [ ]:
# SELECT FROM OPTIONS BELOW OR CREATE YOUR OWN, COMMENT UNUSED:

# basic tesseract (original image size, case insensitive, only alphanumeric)
file_name = "carfttesseract_BD_nosplit_alph"
specifications = ["tesseract untrained", "BD, original image size, case sensitive, special char, nosplit"]

In [ ]:
df_results = pd.DataFrame(list(zip(filenames, iou_in_image, cer_in_image)), columns =['Filename', 'IoU', 'CER'])
mean_iou = round(df_results['IoU'].mean() * 100, 1)
mean_cer = round((1 - df_results['CER'].mean()) * 100, 1)

print(file_name, ": ", specifications[-1])
print(f"mean IoU accuracy = {mean_iou}%, mean CER accuracy = {mean_cer}%")

df_results

### Save results

In [ ]:
# set and create output directory if it doesn't exist

output_dir = 'results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Save to file.

In [ ]:
# create file with results (specify method and other parameters)

with open('%s/result_%s.txt' % (output_dir, file_name), 'w') as output_file:
    output_file.write(": ".join(str(text) for text in specifications))
    output_file.write("\n"+f"iou = {mean_iou}")
    output_file.write("\n"+f"cer = {mean_cer}")

Visualize bounding rectangles and corresponing words.

In [ ]:
# all images / one sample image
# for i in range(n_imgs):
i = 2

im = plot_results(original_images[i], ground_truth[i], predicted[i])  
im.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)

im.savefig('%s/result_%s2.png' % (output_dir, file_name))
im.show()
